# Collect data

This notebooks collects a table with one Wada test result per patient and the filenames of all realigned and normalized fMRI time series (wrf*) of the patients. The information is stored in a table which will be re-loaded and extended in each of the following notebooks.

### import modules

In [1]:
import glob

import numpy as np
import pandas as pd

### get clinical infos

In [ ]:
info_df = pd.read_csv('../data/raw/csv/info_df.csv',index_col=[0])
info_df.tail()

In [3]:
info_df.shape

(101, 2)

### get data

#### fmri timecourses

In [ ]:
im_list = glob.glob('../data/raw/nii/wr*')
im_list.sort()
im_list[-5:]

In [9]:
len(im_list)

143

#### realignment parameters

In [ ]:
rp_list = glob.glob('../data/raw/nii/rp*')
rp_list.sort()
rp_list[-5:]

In [29]:
def make_epi_df(im_list,rp_list):
    """collect all the names of all fmri data in a table"""
    
    epi_dict = {}
    
    for im in im_list:
        p_im_name = im.split('/wrf_')[-1].split('.')[0]
        for rp in rp_list:
                p_rp_name = rp.split('/rp_f_')[-1].split('.')[0]
                if p_im_name == p_rp_name:
                    p_name = p_im_name.split('_')[0]
                    epi_dict[p_im_name] = {'epi':im,'rp':rp,'code':p_name}
                    
    epi_df = pd.DataFrame(epi_dict).T
    epi_df.loc[:,'idx'] = epi_df.index
    
    return epi_df

In [30]:
epi_df = make_epi_df(im_list,rp_list)

In [ ]:
epi_df.tail()

In [32]:
epi_df.shape

(143, 4)

### combine Wada and fMRI data

In [33]:
merge_df = pd.merge(epi_df,info_df,on='code')
merge_df.index = merge_df.loc[:,'idx']
merge_df = merge_df[['code','wada','epi','rp']]

In [ ]:
merge_df.tail()

check for missing values

In [35]:
merge_df.shape, merge_df.dropna().shape

((143, 4), (143, 4))

### how many unique patients are there?

In [43]:
merge_df.groupby('code').count().shape

(98, 3)

### store table

In [36]:
merge_df.to_csv('../data/interim/csv/info_epi_df.csv')

### summary

We now have a table which tells us where all the relevant files for each fMRI sessions are stored and tells us the Wada test results of the patient the data belong to.  


**************

< Previous | [Contents](00-mw-overview-notebook.ipynb) | [Next >](02-mw-make-z-maps.ipynb)